# Practica IA

Utilizaremos la API de `together.ai` para hacer clasificacion de etiquetas.



In [17]:
import os
os.environ["TOGETHER_API_KEY"] =  "tgp_v1_2taO6Uxvh4tv49dnP5wNN6slSiKEvgsQZPXdSPYHnJA"

tgp_v1_2taO6Uxvh4tv49dnP5wNN6slSiKEvgsQZPXdSPYHnJA

## Inferencia con Llama 3.3

In [7]:
#model="meta-llama/Llama-3.2-3B-Instruct-Turbo"
model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"

In [ ]:
pip install together

Prueba de API

In [18]:
from together import Together

client = Together()

response = client.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    messages=[{"role": "user", "content": "What are some fun things to do in New York?"}],
)
print(response.choices[0].message.content)

The city that never sleeps! New York has endless options for entertainment, culture, and adventure. Here are some fun things to do in New York:

**Iconic Landmarks:**

1. Statue of Liberty and Ellis Island: Take a ferry to Liberty Island to see the iconic statue up close and visit the Ellis Island Immigration Museum.
2. Central Park: Explore the park's many walking paths, lakes, and landmarks like the Bethesda Fountain and Loeb Boathouse.
3. Times Square: Experience the bright lights and energy of the "Crossroads of the World."
4. Empire State Building: Enjoy panoramic views of the city from the observation deck on the 86th floor.
5. Brooklyn Bridge: Walk or bike across the iconic bridge for spectacular city views.

**Museums and Galleries:**

1. The Metropolitan Museum of Art: One of the world's largest and most renowned museums, with a collection that spans over 5,000 years of human history.
2. American Museum of Natural History: Explore exhibits on dinosaurs, space, and the natural 

In [19]:
from together import Together

client = Together()

response = client.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    messages=[],
    max_tokens=None,
    temperature=0.7,
    top_p=0.7,
    top_k=50,
    repetition_penalty=1,
    stop=["<|eot_id|>","<|eom_id|>"],
    stream=True
)
for token in response:
    if hasattr(token, 'choices'):
        print(token.choices[0].delta.content, end='', flush=True)

I can be used in a variety of ways, from helping you plan a vacation to creating art. I'm here to assist you in finding the help or information you need. My strengths include answering questions, generating text and images and even just chatting with you.

Texto de etiqueta para  clasificacion

In [20]:
Etiqueta = """
Eau de Parfum
Ingredientes: ver envase
Precauciones: Inflamable. Mantener fuera del alcance de los niños. Mantener en lugar fresco y seco. Sólo para uso externo.
Ver lote y fecha de caducidad en el envase. 
Persona responsable: 
Trending Corporate S.L.C/ Gremios nº5 Benacazón. España. 
 """


In [24]:
prompt = f"""
De cada etiqueta, traduciras a json el texto tomando 
las categorias y su contenido, no expliques como lo haces.
Despues las añadiras a un archivo llamado: "etiquetas.json"
Al añadirlo muestra el mensaje "Producto añadido", si no es posible
indicalo con el mensaje "Error al actualizar la lista".

Las categorias son:
- Nombre
- Ingredientes
- Precauciones
- Persona responsable

Etiqueta: ```{Etiqueta}```
"""

In [25]:
prompt = f"[INST]{prompt}[/INST]"
print(prompt)

[INST]
De cada etiqueta, traduciras a json el texto tomando 
las categorias y su contenido, no expliques como lo haces.
Despues las añadiras a un archivo llamado: "etiquetas.json"
Al añadirlo muestra el mensaje "Producto añadido", si no es posible
indicalo con el mensaje "Error al actualizar la lista".

Las categorias son:
- Nombre
- Ingredientes
- Precauciones
- Persona responsable

Etiqueta: ```
Eau de Parfum
Ingredientes: ver envase
Precauciones: Inflamable. Mantener fuera del alcance de los niños. Mantener en lugar fresco y seco. Sólo para uso externo.
Ver lote y fecha de caducidad en el envase. 
Persona responsable: 
Trending Corporate S.L.C/ Gremios nº5 Benacazón. España. 
 ```
[/INST]


In [26]:
response = client.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    messages=[{"role": "user", "content": prompt}],
)
print(response.choices[0].message.content)

```json
{
  "Nombre": "Eau de Parfum",
  "Ingredientes": "ver envase",
  "Precauciones": "Inflamable. Mantener fuera del alcance de los niños. Mantener en lugar fresco y seco. Sólo para uso externo. Ver lote y fecha de caducidad en el envase.",
  "Persona responsable": "Trending Corporate S.L.C/ Gremios nº5 Benacazón. España."
}
```

Se ha añadido el producto al archivo "etiquetas.json". 
Producto añadido.


## Creamos una función de orquestración
(He intentado adaptarlo de los apuntes a mi modelo pero no termino de entenderlo)

In [ ]:
import os
import warnings
import json
import time
from together import Together


url = "https://api.together.ai/settings/api-keys"

os.environ["TOGETHER_API_KEY"] =  "tgp_v1_2taO6Uxvh4tv49dnP5wNN6slSiKEvgsQZPXdSPYHnJA"

def orchestration(prompt,
                  add_inst=True,
                  model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
                  temperature=0.0,
                  max_tokens=1024,
                  verbose=False,
                  base=2,
                  attempt=0,
                  max_tries=3):

    if add_inst:
        prompt = f"[INST]{prompt}[/INST]"

    if verbose:
        print(f"Prompt:\n{prompt}\n")
        print(f"Model: {model}")

    data = {
            "model": model,
            "prompt": prompt,
            "temperature": temperature,
            "max_tokens": max_tokens
        }

    # Calculate waiting time based on the current attempt number
    wait_seconds = base ** attempt

    try:
        response = client.chat.completions.create(url, headers=headers, json=data)
        if response.status_code == 200:
            return response.json()['output']['choices'][0]['text']
        else:
            raise Exception(f"API response not successful: {response.status_code}")
    except Exception as e:
        if attempt < max_tries - 1:
            print(f"Error message: {e}")
            print(f"Response object: {response}")
            print(f"Attempt {attempt + 1}")
            print(f"Waiting {wait_seconds} seconds before retrying.")
            time.sleep(wait_seconds)
            return orchestration(prompt, add_inst, model, temperature, max_tokens, verbose, base, attempt + 1, max_tries)
        else:
            print(f"Tried {max_tries} times to make API call to get a valid response object.")
            print("Returning the last received response")
            return response

    return response
